# [Sec 0-0] Pytorch trial

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
embedding = nn.Embedding(10, 3)

In [ ]:
embedding.weight

In [ ]:
k=np.random.normal(0, 0.001, (10,3))
embedding.weight = nn.Parameter(torch.from_numpy(k))
embedding.weight

In [ ]:
input = Variable(torch.LongTensor([0,2,4,5]))
input1 = Variable(torch.LongTensor([1]))

In [ ]:
base = embedding(input1)

In [ ]:
a = embedding(input)

In [ ]:
a

## give a try at brocasting

In [ ]:
base * a 

In [ ]:
torch.__version__

In [ ]:
a

# torch matrix multiplication

In [ ]:
torch.t(a)

In [ ]:
print(torch.mm(torch.t(a),a))
print(torch.mm(torch.t(a),a).sum())


In [ ]:
A = torch.mm(torch.t(a),a).float()
W = nn.Parameter(torch.ones(3,3))*2

In [ ]:
print(A)
print(W*A)

In [ ]:
torch.mm(torch.t(a),a).trace()


# [Sec 0-1] Data Pre-processing - hashing

In [ ]:
# reading data 
import pandas as pd
X_train = pd.read_csv("data/train")
X_test = pd.read_csv("data/test")

In [ ]:
# add time column
X_train['weekday']= (X_train['hour']%10000/100).astype(int)%7
X_train['hour_']= (X_train['hour']%100)
X_test['weekday']= (X_test['hour']%10000/100).astype(int)%7
X_test['hour_']= (X_test['hour']%100)

In [ ]:
column2handle = list(X_test.columns.values)[2:]

In [ ]:
print(column2handle)

In [ ]:
hash_list=[]
# will be slowly as list grow up 
def get_hash_value(x):
    try:
        idx = hash_list.index(x)
    except :
        hash_list.append(x)
        idx = len(hash_list)-1
    return idx
    

In [ ]:
# dict version
hash_dict=dict()
count =0 
def get_hash_value_dict(x):
    global count
    idx = hash_dict.get(x,count)
    if idx==count:
        hash_dict[x]=count
        count+=1
    return idx

In [ ]:
column2handle[0]

In [ ]:
X_train.columns.values

### compare 2 implement of hash method

In [ ]:
%%time
i='site_id'
X_train[i+'_hash']=X_train[i].apply(get_hash_value)


In [ ]:
%%time
i='site_id'
X_train[i+'_hash']=X_train[i].apply(get_hash_value_dict)

obviously using dictionary is much suitable here
## build up the hash function

In [ ]:
# dict version
hash_dict=dict()
count =0 
def get_hash_value_dict(x):
    global count
    idx = hash_dict.get(x,count)
    if idx==count:
        hash_dict[x]=count
        count+=1
    return idx

In [ ]:
%%time
for i in column2handle:
    print (i)
    X_train[i+'_hash']=X_train[i].apply(lambda x :get_hash_value_dict(i+'_'+str(x)))


In [ ]:
print("there are total : ",count," category in the training set")
print(count==len(hash_dict.keys()))

In [ ]:
%%time
for i in column2handle:
    print(i)
    X_test[i+'_hash']=X_test[i].apply(lambda x :get_hash_value_dict(i+'_'+str(x)))

In [ ]:
print("there are total : ",count," category in the training& testing set")
print("there are additional : ",count-9449236," category from testing set")
print(count==len(hash_dict.keys()))

In [ ]:
X_train[['click']+[ i+'_hash' for i in column2handle]].to_pickle('train_df_hash')

In [ ]:
X_test[[ i+'_hash' for i in column2handle]].to_pickle('test_df_hash')

In [ ]:
import numpy as np

# Save
np.save('hash_dict.npy', hash_dict) 

# Load
# read_dictionary = np.load('hash_dict.npy').item()


# [Sec1] Building FF with Pytorch

In [ ]:
import pandas as pd 
X_train = pd.read_pickle('train_df_hash')
X_test = pd.read_pickle('test_df_hash')

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
#we need a 10269200 embedding matrix, let k be 100
embedding = nn.Embedding(10269200, 100)

In [ ]:
# give a length 23 array
input = Variable(torch.LongTensor(X_train.iloc[0,1:].values))
# get the corresponds matrix
W = embedding(input)


what we want is 
$$
\phi_{FM}(w,x) =  \Sigma_{j_1,j_2 \epsilon C_{2}}   <w_{j1} , w_{j2}>  * x_{j1} * x_{j2}  -(1)
$$
$w_{j1}$and $w_{j2}$ are two vectors with length k

and our objective function is :
$$
min_{w}  \Sigma_{i=1}^{n} ( log(1+exp(-yi * \phi(w,x_{i}) )) ) + \frac{\lambda}{2} * ||w||^{2} -(2) 
$$

since C2 is a large number : 10269200

for each sample i there acctually will be 23 $w_{j}$ will be involved in (1) 


now we extract the $w_{j}$ with corresponse sample i  have a matrix which is 
$$
W_{i} = \left(\begin{array}{cc} 
w_{11} & ... w_{1k}\\
w_{21} & ... w_{2k}\\
...\\
w_{f1} & ... w_{fk}\\
\end{array}\right)
$$ 

Let
$$
A = W_{i} * W_{i}'
$$

therefore
$$
\phi_{FM}(w,x_{i}) =  (sum(A) + trace(A))/2
$$


In [ ]:
W.size()

In [ ]:
A = torch.mm(W,torch.t(W))
A.sum()+ A.trace()

In [ ]:
X_train['click'][0]

# [Sec1-1] now make a nn.module for FM

In [1]:
import pandas as pd 
X_train = pd.read_pickle('train_df_hash')
X_test = pd.read_pickle('test_df_hash')

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
# create dataloader
msk = np.random.rand(len(X_train)) < 0.8
train_data = torch.LongTensor(X_train.iloc[msk,1:].values)
train_labels = torch.from_numpy(X_train.iloc[msk,0].values).float()

test_data = torch.LongTensor(X_train.iloc[~msk,1:].values)
test_labels = torch.from_numpy(X_train.iloc[~msk,0].values).float()


In [3]:
# create our dataloader first 
import torch.utils.data as data_utils

train = data_utils.TensorDataset(train_data, train_labels)
train_loader = data_utils.DataLoader(train, batch_size=1, shuffle=True,num_workers=8)

test = data_utils.TensorDataset(test_data, test_labels)
test_loader = data_utils.DataLoader(test, batch_size=1, shuffle=False,num_workers=8)

In [ ]:
class FF(nn.Module):
    def __init__(self,k=50):
        super(FF, self).__init__()
        self.embedding = nn.Embedding(10269200, k, sparse=True)
        init_mat = np.random.normal(0, 0.0001, (10269200,k)) # default embedding variance too large
        self.embedding.weight = nn.Parameter(torch.from_numpy(init_mat).float())
        self.bias = nn.Parameter(torch.ones(1)*0.18606) # let initial prob is close to 0.1698(mean of train click)
        self.D = nn.Parameter(torch.ones(23,23))
        self.dropout = nn.Dropout(p=0.5)
    def forward(self, x):
        W = self.embedding(x[0])
        A = torch.mm(W,torch.t(W)) * self.D
        A = self.dropout(A)
        return (A.sum()+ A.trace()).view(-1, 1)+self.bias

In [ ]:
class StableBCELoss(torch.nn.modules.Module):
    def __init__(self):
        super(StableBCELoss, self).__init__()
    def forward(self, input, target):
        neg_abs = - input.abs()
        loss = input.clamp(min=0) - input * target + (1 + neg_abs.exp()).log()
        return loss.mean()

In [6]:
import torch.optim as optim
net = FF()
net.cuda()
criterion = StableBCELoss()

In [ ]:
optimizer = optim.Adagrad(net.parameters(), lr=1e-2)

In [ ]:
#net.load_state_dict(torch.load('FF_epoch{}.pth'.format(1)))

In [ ]:
min_testing_loss = 100
#min_testing_loss = 0
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    net.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda()), Variable(labels.float().cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 10000 == 9999:    # print every 10000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10000))
            running_loss = 0.0
            
    #============= eval testing ================
    running_loss = 0.0
    count=0
    net.eval()
    for i, data in enumerate(test_loader, 0):
        count+=1
        inputs, labels = data
        inputs, labels = Variable(inputs.cuda()), Variable(labels.float().cuda())
        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        # print statistics
        running_loss += loss.data[0]
    testing_loss = running_loss / count
    print('[%d, %5d] testing loss: %.3f' %(epoch + 1, count, testing_loss))
    if min_testing_loss>testing_loss:
        torch.save(net.state_dict(), 'FF_epoch{}_{}.pth'.format(epoch,testing_loss))
        min_testing_loss = testing_loss
    running_loss = 0.0
print('Finished Training')

[1, 10000] loss: 0.559
[1, 20000] loss: 0.464
[1, 30000] loss: 0.417
[1, 40000] loss: 0.423
[1, 50000] loss: 0.436
[1, 60000] loss: 0.415
[1, 70000] loss: 0.418
[1, 80000] loss: 0.422
[1, 90000] loss: 0.420
[1, 100000] loss: 0.408
[1, 110000] loss: 0.413
[1, 120000] loss: 0.413
[1, 130000] loss: 0.421
[1, 140000] loss: 0.421
[1, 150000] loss: 0.414
[1, 160000] loss: 0.407
[1, 170000] loss: 0.422
[1, 180000] loss: 0.405
[1, 190000] loss: 0.413
[1, 200000] loss: 0.407
[1, 210000] loss: 0.401
[1, 220000] loss: 0.418
[1, 230000] loss: 0.414
[1, 240000] loss: 0.419
[1, 250000] loss: 0.401
[1, 260000] loss: 0.401
[1, 270000] loss: 0.405
[1, 280000] loss: 0.404
[1, 290000] loss: 0.401
[1, 300000] loss: 0.417
[1, 310000] loss: 0.401
[1, 320000] loss: 0.412
[1, 330000] loss: 0.400
[1, 340000] loss: 0.415
[1, 350000] loss: 0.408
[1, 360000] loss: 0.406
[1, 370000] loss: 0.399
[1, 380000] loss: 0.413
[1, 390000] loss: 0.403
[1, 400000] loss: 0.408
[1, 410000] loss: 0.401
[1, 420000] loss: 0.405
[

In [ ]:
torch.save(net.state_dict(), 'FF_epoch{}_{}.pth'.format(1,min_testing_loss))

In [ ]:
#[1, 8084727] testing loss: 0.375

# torch.save(net.state_dict(), 'FF_epoch{}_{}.pth'.format(1,0.373))


# [Sec1-2] apply on testing Set

In [ ]:
def prob(x):
    a=np.exp(x)
    return a/(1+a)

In [ ]:
#net.load_state_dict(torch.load('FF_epoch{}.pth'.format(1)))

A = np.array([])
for i in range(len(X_test)):
    input = X_test.values[i]
    input = Variable(torch.LongTensor(input))
    input = input.contiguous().view(1,-1).cuda()
    out = net(input)
    prob_out = prob(out.data.cpu().numpy()[0][0])
    A = np.append(A, prob_out)

In [ ]:
import pandas as pd
sampleSubmission = pd.read_csv("data/sampleSubmission")
sampleSubmission.click = A
# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("FF_pytorch_20170826.csv",index=False)

# [Sec2] Build a model for Batch processing

In [ ]:
# with using torch.bmm

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
embedding = nn.Embedding(10, 3)

In [ ]:
print(torch.mm(torch.t(a),a))
print(torch.mm(torch.t(a),a).sum())
